In [ ]:
# imports
import numpy as np
import pandas as pd
import requests
import spotipy
import librosa
import json

from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup

In [ ]:
# spotify api creds
SPOTIFY_CLIENT_ID = ''
SPOTIFY_CLIENT_SECRET = ''

In [ ]:
# function to get spotify token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']

In [ ]:
# track details function
def get_track_details(spotify_id, token):
    sp = spotipy.Spotify(auth=token)
    track = sp.track(spotify_id)
    return track['name'], track['artists'][0]['name']

In [ ]:
# lyrics by spotify id
def get_lyrics_by_spotify_id(spotify_id):
    # spotify token
    spotify_token = get_spotify_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)

    song_name, artist_name = get_track_details(spotify_id, spotify_token)
    
    song_name = str(song_name)
    artist_name = str(artist_name)

    url = 'https://api.lyrics.ovh/v1/' + artist_name + '/' + song_name

    response = requests.get(url)
    lyrics = json.loads(response.content)   

    return lyrics

In [ ]:
# example usage (the middle - zedd)
spotify_id = 'https://open.spotify.com/track/09IStsImFySgyp0pIQdqAc?si=31abcdbe97a641b5'
lyrics = get_lyrics_by_spotify_id(spotify_id)

print(lyrics)